In [1]:
import pandas as pd
import Levenshtein

In [2]:
df_house = pd.read_csv('/Users/william/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/housingquery.csv', dtype={'PrimaryID': str})

In [3]:
pd.options.display.max_colwidth = 300
pd.options.display.max_rows = 300

## Read and format cdbg data

In [4]:
lines = []
for i in range(2007, 2018):
    file = '/Users/william/Documents/Summer2019/AlbanyHousing/Raw Data/CED/parse/cdbg' + str(i) + '.csv'
    with open(file, 'r') as f:
        lines.extend(f.readlines())

In [5]:
length = len(lines)

### Most of this code is just cleaning and splitting to find relevant info. 

variable names and such will be explained below

In [6]:
# split each project into separate parcel
start = 0
end = 0
parcels = []
i = 0
while i < length:
    try:
        start = end
        i += 1
        while "PGM Year" not in lines[i]:
            i += 1
        end = i
        parcel = lines[start:end]
        parcels.append(parcel)
    except:
        print("done")
        break

done


In [7]:
parcels[0]

['\ufeffPGM Year:,,,,2005,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'Project:,,,,0009 - PUBLIC SERVICE                                              ,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'IDIS Activity:,,,,445 - CITY OF ALBANY HOMELESS PROGRAM,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 ',,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'Status:,,,,Completed 6/30/2006 12:00:00 AM,,,,,,,,,,,,,,,,,,,,,Objective:,,,,,Provide decent affordable housing,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'Location:,,,,"230 S Jackson St                                                                                             Albany, GA  31701-2885",,,,,,,,,,,,,,,,,,,,,Outcome:,,,,,Affordability,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 ',,,,,,,,,,,,,,,,,,,,,,,,,Matrix Code:,,,,,Operating Costs of Homeless/AIDS Patients Programs (03T),,,,,,,,,,,National Objective:,,,,,,,,,,,LMC,,,,,\n',
 ',,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'Initial

## Fields that we want to pull

In [7]:
# split each parcel into smaller bits of data with relevant info
condensedparcels = []
for parcel in parcels:
    condensed = []
    for i in range(len(parcel)):
        if "Project" in parcel[i]:
            condensed.append(parcel[i])
        if "IDIS Activity" in parcel[i]:
            condensed.append(parcel[i])
        if "Status" in parcel[i]:
            condensed.append(parcel[i])
        if "Location" in parcel[i]:
            condensed.append(parcel[i])
        if "Description" in parcel[i]:
            condensed.append("Description:" + parcel[i+1])
    #for line in parcel:
    #    if "IDIS Activity" in line:
    #        condensed.append(line)
    #    if "Status" in line:
    #        condensed.append(line)
    #    if "Location" in line:
    #        condensed.append(line)
    condensedparcels.append(condensed)

In [8]:
condensedparcels[8]

['Project:,,,,0006 - EMERGENCY REPAIRS                                           ,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'IDIS Activity:,,,,506 - 916 CEDAR AVENUE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'Status:,,,,Completed 9/7/2007 12:00:00 AM,,,,,,,,,,,,,,,,,,,,,Objective:,,,,,Provide decent affordable housing,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'Location:,,,,"916 Cedar Ave                                                                                             Albany, GA  31701-2945",,,,,,,,,,,,,,,,,,,,,Outcome:,,,,,Affordability,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'Description:PERFORM EMERGENCY REPAIRS TO A SINGLE-FAMILY OWNER-OCCUPIED UNIT.,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n']

#### formatting

In [9]:
# format to a parseable piece
totalformatted = []
for i in range(len(condensedparcels)):
    formatted = []
    proj = condensedparcels[i][0].strip("\n").strip(",").split(",")
    formatted.append(proj[0] + ":" + proj[-1])
    idis = condensedparcels[i][1].split("-")[0].replace(",","").replace('"',"")
    status = condensedparcels[i][2].replace(",","").split()[0:2]
    location = condensedparcels[i][3].replace(",","").split('"')[0:2]
    formatted.append(idis.strip())
    formatted.append(status[0] + ":" + status[1])
    desc = condensedparcels[i][4].strip("\n").strip(",")
    try:
        formatted.append(location[0] + location[1])
    except:
        firstindex = 9
        secondindex = location[0].find("Outcome")
        #secondindex = location[0].split(":")[1].find("Outcome")
        #formatted.append(location[0].split(":")[1])
        formatted.append("Location:" + location[0][firstindex:secondindex])
        print("location " + location[0] + " not found")
    formatted.append(desc)
    totalformatted.append(formatted)

location Status:Completed 10/31/2010 12:00:00 AMObjective:Create suitable living environments
 not found
location Status:Completed 11/20/2013 12:00:00 AMObjective:Create suitable living environments
 not found
location Status:Completed 10/31/2010 12:00:00 AMObjective:Create suitable living environments
 not found
location Status:Completed 11/20/2013 12:00:00 AMObjective:Create suitable living environments
 not found
location Status:Completed 11/20/2013 12:00:00 AMObjective:Create suitable living environments
 not found
location Location:Address SuppressedOutcome:Availability/accessibility
 not found
location Status:Completed 11/20/2013 12:00:00 AMObjective:Create suitable living environments
 not found
location Location:Address SuppressedOutcome:Availability/accessibility
 not found
location Status:Completed 11/20/2013 12:00:00 AMObjective:Create suitable living environments
 not found
location Location:Address SuppressedOutcome:Availability/accessibility
 not found
location Status:Com

In [10]:
totalformatted[314]

['Project::0015 - PUBLIC SERVICE',
 'IDIS Activity:774',
 'Status:Completed:10/31/2010',
 'Location:PO Box 44   Albany GA  31702-0044',
 'Description:To expand the service of assessing individuals served to determine needs; and to enhance the services of the soup kitchen.']

In [11]:
# slightly more formatting
proj_list = []
idisno_list = []
status_list = []
status_date_list = []
location_list = []
desc_list = []
for project in totalformatted:
    proj_list.append(project[0].split(":")[-1])
    idisno_list.append(project[1].split(":")[1])
    status_full = project[2].split(":")
    status_list.append(status_full[1])
    status_date_list.append(status_full[2])
    location_list.append(project[3].split(":")[1].strip())
    desc_list.append(project[4].split(":")[1])

In [12]:
# renaming columns
df = pd.DataFrame({'Project': proj_list, 'Activity': idisno_list, 'Status': status_list, 
                   'Status Date': status_date_list, 'Address': location_list, 'Description': desc_list})

# Start: df is name of dataframe.

### descriptions may need to be cleaned up a bit more also. Can pull in more fields if necessary from the pr03s. The thing that may be useful is pulling in the objective.

In [13]:
df.head()

,Project,Activity,Status,Status Date,Address,Description
0,0009 - PUBLIC SERVICE,445,Completed,6/30/2006,230 S Jackson St Albany GA 31701-2885,"""TO HELP THE HOMELESS IN ALBANY, GEORGIA"""
1,0006 - HOUSING,466,Canceled,1/10/2012,SANDS DRIVE ALBANY GA 31705,"""THE ACQUISITION OF PROPERTY FOR THE DEVELOPMENT OF LOW ANDMODERATE INCOME HOUSING."
2,0001 - ADMINISTRATION,473,Completed,6/30/2007,230 S Jackson St SUITE 315 Albany GA 31701-2885,"""THE OPERATION OF THE CDBG PROGRAM."
3,0002 - SECTION 108 LOAN,475,Completed,8/25/2007,230 S Jackson St SUITE 315 Albany GA 31701-2885,"""THE SECTION 108 PAYMENT FOR THE LOAN RELATED TO THE HILTONGARDEN PROJECT."
4,0002 - SECTION 108 LOAN,476,Completed,8/25/2007,230 S Jackson St SUITE 315 Albany GA 31701-2885,THE REPAYMENT OF A SECTION 108 LOAN FOR THE MCGREGOR PROJECT


In [122]:
df[df['Description'].str.upper().str.contains("WINDOWS")]

,Project,Activity,Status,Status Date,Address,Description
402,0005 - Energy Efficiency,821,Completed,9/27/2010,3005 Harvest Ln Albany GA 31721-4514,"""Energy Efficiency measures were installed to include 12 windows, and 2 new exterior doors."""
403,0005 - Energy Efficiency,822,Canceled,2/10/2011,103 Almond Dr Albany GA 31705-2528,Energy Efficiency measures were installed to include 11 windows.
404,0005 - Energy Efficiency,823,Completed,3/2/2011,706 Easttown Rd Albany GA 31705-1214,"""Energy Efficiency measures were installed to include 13 windows, and 2 new exterior doors."""
405,0005 - Energy Efficiency,824,Completed,9/30/2010,919 W Lincoln Ave Albany GA 31701-2919,"""Energy Efficiency measures were installed to includewindows, andnew exterior doors."""
406,0005 - Energy Efficiency,825,Completed,2/14/2011,2004 Colquitt Ave Albany GA 31707-5129,"""Energy Efficiency measures were installed to include 13 windows, and 2 new exterior doors."""
408,0005 - Energy Efficiency,827,Completed,2/14/2011,2514 Whispering Pines Rd Albany GA 31707-2234,"""Energy Efficiency measures were installed to include15 windows, and 1 new exterior doors."""
409,0005 - Energy Efficiency,829,Completed,2/2/2011,1530 E Broad Ave Albany GA 31705-2210,Energy Efficiency measures were installed to include 19 windows.
410,0005 - Energy Efficiency,830,Completed,2/14/2011,2230 S Washington St Albany GA 31701-3735,"""Energy Efficiency measures were installed to include 11 windows, and2 new exterior doors."""
413,0005 - Energy Efficiency,833,Completed,4/28/2011,1105 Jordan St Albany GA 31705-1371,Energy Efficiency measures were installed to includewindows.
415,0005 - Energy Efficiency,835,Canceled,2/9/2011,402 Bush St Albany GA 31705-4004,Energy Efficiency measures were installed to include 14 windows.


## Everything below is pulling in HOME files and standardizing addresses. Would recommend copying in everything between start and here and copying into a separate file.

In [14]:
# s_addresses contains all the addresses in the pr03 files
s_addresses = df['Address']
for i in range(len(s_addresses)):
    words = s_addresses[i].split()
    index = 0
    for j in range(len(words)):
        if words[j] == "Albany" or words[j] == "ALBANY":
            index = j
    words = words[0:index]
    s_addresses[i] = words

In [15]:
s_addresses.head()

0                [230, S, Jackson, St]
1                       [SANDS, DRIVE]
2    [230, S, Jackson, St, SUITE, 315]
3    [230, S, Jackson, St, SUITE, 315]
4    [230, S, Jackson, St, SUITE, 315]
Name: Address, dtype: object

In [16]:
# some format/capitalization/casting for the addresses
for i in range(len(s_addresses)):
    for j in range(len(s_addresses[i])):
        if not s_addresses[i][j].isnumeric():
            s_addresses[i][j] = s_addresses[i][j].upper()

In [17]:
s_addresses.head()

0                [230, S, JACKSON, ST]
1                       [SANDS, DRIVE]
2    [230, S, JACKSON, ST, SUITE, 315]
3    [230, S, JACKSON, ST, SUITE, 315]
4    [230, S, JACKSON, ST, SUITE, 315]
Name: Address, dtype: object

In [18]:
df_house = df_house[df_house['Program'] == 'CDBG']

In [19]:
df.shape

(1110, 4)

In [20]:
# have to reformat address so that we pd can hash to drop duplicates
for i in range(len(df)):
    addr = df['Address'][i]
    word = ""
    for pt in addr:
        word += pt + " "
    word = word.strip()
    df['Address'][i] = word

In [21]:
df = df.drop_duplicates()

In [22]:
# might help with matching
def lev_list(address_list, address,i):
    temp_list = []
    temp_list_u = []
    for u_address in address_list:
        if Levenshtein.distance(address, u_address.upper()) <= i:
            temp_list.append(address)
            temp_list_u.append(u_address.upper())
    temp_list_series = pd.Series(temp_list)
    temp_list_u_series = pd.Series(temp_list_u)
    return temp_list_u_series

#### df is the dataframe with completed date
#### df_house is the dataframe with housing projects

In [23]:
df_house.index = range(len(df_house))

In [24]:
dates = []
addresses = []
for i in range(len(df_house)):
    completed_addresses = df[str(df_house['IDISActivityID'][i]) == df['Activity']]
    if len(completed_addresses) == 1:
        completed_addresses.index = range(len(completed_addresses))
        date = completed_addresses['Status Date'][0]
        addr = completed_addresses['Address'][0] # for checking purposes
        if completed_addresses['Status'][0] == "Open":
            dates.append("Continuing")
        else:
            dates.append(date)
        addresses.append(addr)
    elif len(completed_addresses) == 0:
        dates.append("NA")
    elif len(completed_addresses) == 2:
        print("multiple")

In [25]:
addresses[2]

'305 S CARROLL ST'

In [26]:
dates = pd.Series(dates)

In [27]:
dates[0]

'6/30/2008'

In [28]:
df_house.head()

,PrimaryID,ParcelID,PlanYear,IDISProject,Project,IDISActivityID,ActivityStatus,Program,FundedAmount,DrawAmount,Balance,MultihomeType,Multihome,Address
0,28,NaN,2007,4,Economic Development,628,Completed,CDBG,2156.32,2156.32,0.0,NaN,NaN,1706 S JEFFERSON ST
1,718,NaN,2014,9,Emergency Repairs,1261,Completed,CDBG,5060.00,5060.00,0.0,NaN,NaN,300 S CARROLL ST
2,720,NaN,2016,5,Emergency Repairs,1473,Completed,CDBG,12061.31,12061.31,0.0,NaN,NaN,305 S CARROLL ST
3,750,NaN,2011,6,Emergency Repairs,1022,Completed,CDBG,11378.50,11378.50,0.0,NaN,NaN,1405 E GORDON AVE
4,753,NaN,2015,19,Emergency Repairs,1362,Completed,CDBG,5554.35,5554.35,0.0,NaN,NaN,1402 E GORDON AVE


In [29]:
dates[57]

'11/30/2014'

In [30]:
df[df['Address'] == '610 W LINCOLN AVE']

,Activity,Status,Status Date,Address
836,1235,Completed,11/30/2014,610 W LINCOLN AVE


In [31]:
df_house[df_house['IDISActivityID'] == 1235]

,PrimaryID,ParcelID,PlanYear,IDISProject,Project,IDISActivityID,ActivityStatus,Program,FundedAmount,DrawAmount,Balance,MultihomeType,Multihome,Address
57,4379,NaN,2014,9,Emergency Repairs,1235,Completed,CDBG,14790.0,14790.0,0.0,NaN,NaN,610 LINCOLN AVE


In [32]:
parcels[836]

['PGM Year:,,,,2014,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'Project:,,,,0009 - Emergency Repairs,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'IDIS Activity:,,,,1235 - 610 Lincoln,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 ',,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'Status:,,,,Completed 11/30/2014 12:00:00 AM,,,,,,,,,,,,,,,,,,,,,Objective:,,,,,Provide decent affordable housing,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'Location:,,,,"610 W Lincoln Ave   Albany, GA  31701-3110",,,,,,,,,,,,,,,,,,,,,Outcome:,,,,,Affordability,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 ',,,,,,,,,,,,,,,,,,,,,,,,,Matrix Code:,,,,,Rehab; Single-Unit Residential (14A),,,,,,,,,,,National Objective:,,,,,,,,,,,LMH,,,,,\n',
 ',,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'Initial Funding Date:,,,,,,,,,,,10/14/2014,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 'Description:,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 '"Emergency r

In [33]:
df.shape

(685, 4)

In [34]:
dates.shape

(564,)

In [35]:
df_house.head()

,PrimaryID,ParcelID,PlanYear,IDISProject,Project,IDISActivityID,ActivityStatus,Program,FundedAmount,DrawAmount,Balance,MultihomeType,Multihome,Address
0,28,NaN,2007,4,Economic Development,628,Completed,CDBG,2156.32,2156.32,0.0,NaN,NaN,1706 S JEFFERSON ST
1,718,NaN,2014,9,Emergency Repairs,1261,Completed,CDBG,5060.00,5060.00,0.0,NaN,NaN,300 S CARROLL ST
2,720,NaN,2016,5,Emergency Repairs,1473,Completed,CDBG,12061.31,12061.31,0.0,NaN,NaN,305 S CARROLL ST
3,750,NaN,2011,6,Emergency Repairs,1022,Completed,CDBG,11378.50,11378.50,0.0,NaN,NaN,1405 E GORDON AVE
4,753,NaN,2015,19,Emergency Repairs,1362,Completed,CDBG,5554.35,5554.35,0.0,NaN,NaN,1402 E GORDON AVE


In [36]:
df_house.iloc[260]

PrimaryID                             20863
ParcelID                                NaN
PlanYear                               2017
IDISProject                              22
Project           Commercial Rehabilitation
IDISActivityID                         1547
ActivityStatus                         Open
Program                                CDBG
FundedAmount                              0
DrawAmount                                0
Balance                                   0
MultihomeType                           NaN
Multihome                               NaN
Address                        514 PINE AVE
Name: 260, dtype: object

In [37]:
addresses[260]

'1118 W WHITNEY AVE'

In [38]:
dates[dates == "NA"]

260    NA
299    NA
dtype: object

In [39]:
dates[dates == "Continuing"]

17     Continuing
26     Continuing
42     Continuing
58     Continuing
60     Continuing
79     Continuing
92     Continuing
183    Continuing
193    Continuing
215    Continuing
226    Continuing
258    Continuing
264    Continuing
277    Continuing
315    Continuing
411    Continuing
412    Continuing
415    Continuing
416    Continuing
417    Continuing
419    Continuing
420    Continuing
445    Continuing
496    Continuing
531    Continuing
534    Continuing
535    Continuing
536    Continuing
537    Continuing
538    Continuing
539    Continuing
540    Continuing
541    Continuing
542    Continuing
543    Continuing
544    Continuing
545    Continuing
546    Continuing
dtype: object

In [40]:
df_house.iloc[26]

PrimaryID                      2092
ParcelID                        NaN
PlanYear                       2014
IDISProject                      11
Project           Public Facilities
IDISActivityID                 1503
ActivityStatus                 Open
Program                        CDBG
FundedAmount                  12500
DrawAmount                    12500
Balance                           0
MultihomeType                   NaN
Multihome                       NaN
Address                603 PINE AVE
Name: 26, dtype: object

In [41]:
df[df['Activity'] == 1235]

,Activity,Status,Status Date,Address


#### Reload in dataset to fix home grants with different formatting

In [42]:
df_house_home = pd.read_csv('/Users/william/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/housingquery.csv', dtype={'PrimaryID': str})

In [43]:
df_home = pd.read_csv('/Users/william/Documents/Summer2019/AlbanyHousing/Raw Data/CED/parse/homefull.csv')

In [44]:
df_home.tail()

,Tenure Type,Activity Type,Grantee,IDIS Activity,Activity Address,Activity Status,Status Date,Total Units,Home Units,Initial Funding Date,Committed Amount,Drawn Amount,PCT
377,Tenant-Based Rental Assistance (TBRA),TENANT-BASED RENTAL ASSISTANCE,39508,1531,", ,",Open,5/18/18,0,1,12/28/17,"$6,166.00","$2,879.59",46.70%
378,Tenant-Based Rental Assistance (TBRA),TENANT-BASED RENTAL ASSISTANCE,39508,1532,", ,",Open,5/18/18,0,1,1/2/18,"$6,917.00","$3,341.59",48.31%
379,Tenant-Based Rental Assistance (TBRA),TENANT-BASED RENTAL ASSISTANCE,39508,1538,", ,",Open,5/18/18,0,1,2/2/18,"$3,520.00","$2,256.91",64.12%
380,Tenant-Based Rental Assistance (TBRA),TENANT-BASED RENTAL ASSISTANCE,39508,1539,", ,",Open,5/18/18,0,1,2/26/18,"$7,805.20","$1,704.08",21.83%
381,Tenant-Based Rental Assistance (TBRA),TENANT-BASED RENTAL ASSISTANCE,39508,1540,", ,",Open,5/18/18,0,1,2/27/18,"$3,547.20","$1,322.94",37.30%


In [45]:
df_home['Activity Address'] = df_home['Activity Address'].str.split(" , ", expand = True)[0]

In [46]:
df_home.tail()

,Tenure Type,Activity Type,Grantee,IDIS Activity,Activity Address,Activity Status,Status Date,Total Units,Home Units,Initial Funding Date,Committed Amount,Drawn Amount,PCT
377,Tenant-Based Rental Assistance (TBRA),TENANT-BASED RENTAL ASSISTANCE,39508,1531,,Open,5/18/18,0,1,12/28/17,"$6,166.00","$2,879.59",46.70%
378,Tenant-Based Rental Assistance (TBRA),TENANT-BASED RENTAL ASSISTANCE,39508,1532,,Open,5/18/18,0,1,1/2/18,"$6,917.00","$3,341.59",48.31%
379,Tenant-Based Rental Assistance (TBRA),TENANT-BASED RENTAL ASSISTANCE,39508,1538,,Open,5/18/18,0,1,2/2/18,"$3,520.00","$2,256.91",64.12%
380,Tenant-Based Rental Assistance (TBRA),TENANT-BASED RENTAL ASSISTANCE,39508,1539,,Open,5/18/18,0,1,2/26/18,"$7,805.20","$1,704.08",21.83%
381,Tenant-Based Rental Assistance (TBRA),TENANT-BASED RENTAL ASSISTANCE,39508,1540,,Open,5/18/18,0,1,2/27/18,"$3,547.20","$1,322.94",37.30%


In [47]:
df_house_home = df_house_home[df_house_home['Program'] == "HOME"]

In [48]:
df_house_home.index = range(len(df_house_home))

In [49]:
df_house_home['IDISActivityID'][0]

1125

In [50]:
df_home.head()

,Tenure Type,Activity Type,Grantee,IDIS Activity,Activity Address,Activity Status,Status Date,Total Units,Home Units,Initial Funding Date,Committed Amount,Drawn Amount,PCT
0,Rental,NEW CONSTRUCTION,39508,28,720 Bonny View Ave,Completed,6/1/96,4,4,5/8/95,"$171,421.03","$171,421.03",100.00%
1,Rental,NEW CONSTRUCTION,39508,178,302 Adkins St,Completed,7/2/97,13,4,7/1/97,"$23,987.70","$23,987.70",100.00%
2,Rental,NEW CONSTRUCTION,39508,285,700 N Davis St,Completed,4/1/02,4,4,5/10/01,"$281,081.26","$281,081.26",100.00%
3,Rental,NEW CONSTRUCTION,39508,286,215 N Broadway St,Completed,8/1/04,24,24,5/10/01,"$1,804,882.55","$1,804,882.55",100.00%
4,Rental,NEW CONSTRUCTION,39508,367,"208-212 SOUTH MADISON AND 506, 527 HIGHLAND",Completed,11/1/04,6,6,5/31/03,"$534,569.40","$534,569.40",100.00%


In [51]:
dates_home = []
addresses_home = []
for i in range(len(df_house_home)):
    completed_addresses_home = df_home[df_house_home['IDISActivityID'][i] == df_home['IDIS Activity']]
    if len(completed_addresses_home) == 1:
        completed_addresses_home.index = range(len(completed_addresses_home))
        date_home = completed_addresses_home['Status Date'][0]
        addr_home = completed_addresses_home['Activity Address'][0] # for checking purposes
        if completed_addresses_home['Activity Status'][0] == "Open":
            dates_home.append("Continuing")
        else:
            dates_home.append(date_home)
        addresses_home.append(addr_home)
    elif len(completed_addresses_home) == 0:
        dates_home.append("NA")
    elif len(completed_addresses_home) == 2:
        print("multiple")

In [52]:
dates_home

['9/2/16',
 'NA',
 'NA',
 '3/9/15',
 '9/2/16',
 '9/29/16',
 'NA',
 '12/3/15',
 '2/5/14',
 'NA',
 '12/3/15',
 '4/3/15',
 '9/14/12',
 '6/10/13',
 '2/5/14',
 '10/27/17',
 'Continuing',
 '12/31/13',
 '2/1/13',
 '4/3/15',
 '10/22/13',
 '10/23/13',
 '4/3/15',
 '3/3/16',
 '10/22/13',
 '10/22/13',
 '10/22/13',
 '10/22/13',
 '10/22/13',
 '10/24/13',
 '9/6/13',
 '12/13/13',
 '10/27/17',
 '9/29/16',
 '9/29/16',
 '12/16/14',
 '9/16/14',
 '9/29/16',
 '9/29/16',
 '9/29/16',
 '9/29/16',
 '9/29/16',
 '9/29/16',
 '3/10/17',
 '7/14/16',
 '9/1/16',
 '3/3/16',
 '3/3/16',
 '7/14/16',
 '7/27/11',
 '4/3/15',
 '8/4/17',
 'Continuing',
 '3/3/16',
 '3/28/17',
 '9/2/16',
 '11/30/17',
 '3/9/15',
 '4/3/15',
 'Continuing',
 'Continuing',
 '4/10/12',
 '9/29/16',
 '6/14/13',
 '9/19/13',
 '7/14/16',
 '4/26/18',
 '6/22/11',
 'NA',
 '11/23/11',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 'NA',
 '2/5/14',
 '11/23/11',
 '5/30/18',
 '4/3/15',
 '2/1/13',
 '2/1/13',
 '12/9/11',
 '8/21/13',
 '7/31/11',
 'Continuing',
 '

In [53]:
dates_home[1]

'NA'

In [54]:
addresses[1]

'300 S CARROLL ST'

In [55]:
df_house_home.loc[0]

PrimaryID                         68
ParcelID             0000C/00005/027
PlanYear                        2012
IDISProject                       20
Project             New Construction
IDISActivityID                  1125
ActivityStatus             Completed
Program                         HOME
FundedAmount                  158828
DrawAmount                    158828
Balance                            0
MultihomeType                    NaN
Multihome                        NaN
Address           252 BONNY VIEW AVE
Name: 0, dtype: object